In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import fnmatch
import cv2
import numpy as np
import string
import time
import sys

In [ ]:
char_list = string.ascii_letters+string.digits

def encode_to_labels(txt):
    dig_lst = []
    for index, char in enumerate(txt):
        try:
            dig_lst.append(char_list.index(char))
        except:
            print(char)

    return dig_lst

In [ ]:
import tarfile
path= '/content/drive/MyDrive/mjsynth.tar.gz'
with tarfile.open(path, 'r:gz') as tar:
  training_img = np.zeros((390000, 32, 128, 1),dtype='uint8')
  training_txt = []
  train_input_length = np.zeros((390000, ),dtype='uint8')
  train_label_length = np.zeros((390000, ),dtype='uint8')
  max_label_len = 0
  i =0
  flag = 0
  for member in tar:
    if fnmatch.fnmatch(member.name, '*.jpg'):
      img = cv2.imdecode(np.frombuffer(tar.extractfile(member).read(), np.uint8), cv2.IMREAD_GRAYSCALE)
      for root, dirnames, filenames in os.walk(path):
        for f_name in fnmatch.filter(filenames, '*.jpg'):
          try:
            img = cv2.cvtColor(cv2.imread(os.path.join(root, f_name)), cv2.COLOR_BGR2GRAY)
            w, h = img.shape
            if h > 128 or w > 32:
              continue
            if w < 32:
              add_zeros = np.ones((32-w, h))*255
              img = np.concatenate((img, add_zeros))
            if h < 128:
              add_zeros = np.ones((32, 128-h))*255
              img = np.concatenate((img, add_zeros), axis=1)
            img = np.expand_dims(img , axis = 2)
            txt = f_name.split('_')[1]
            if len(txt) > max_label_len:
              max_label_len = len(txt)
            if len(txt) == 0:
              print(txt)
              continue
            train_label_length[i]= len(txt)
            train_input_length[i]= 31
            training_img[i]= img
            training_txt.append(encode_to_labels(txt))

            sys.stdout.write('\r' + str(i) + ' ' + str(np.count_nonzero(train_label_length)))

            if i == 389999:
              flag = 1
              break
            i+=1

          except:
            print('\n' + os.path.join(root, f_name))
        if flag == 1:
          break

train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))

error: ignored

In [ ]:
train_label_length = np.array(train_label_length, dtype='uint8')
train_input_length = np.array(train_input_length, dtype='uint8')
training_img = np.array(training_img, dtype='uint8')
train_padded_txt = np.array(train_padded_txt,dtype='uint8')

In [ ]:
print("Number of training examples:", len(training_txt))
print("Lengths of original labels:", [len(txt) for txt in training_txt])
print("Max label length:", max_label_len)

Number of training examples: 0
Lengths of original labels: []
Max label length: 0


In [ ]:
print(train_label_length.shape)
print(train_input_length.shape)
print(training_img.shape)
print(train_padded_txt.shape)

(390000,)
(390000,)
(390000, 32, 128, 1)
(0, 0)


In [ ]:
np.save('/content/drive/My Drive/train_label_length.npy',train_label_length)
np.save('/content/drive/My Drive/train_input_length.npy',train_input_length)
np.save('/content/drive/My Drive/training_img.npy',training_img)
np.save('/content/drive/My Drive/train_padded_txt.npy',train_padded_txt)
np.save('/content/drive/My Drive/max_label_len.npy',max_label_len)

In [ ]:
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional,Dropout
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import keras

In [ ]:
train_label_length = np.load('/content/drive/My Drive/train_label_length.npy')
train_input_length = np.load('/content/drive/My Drive/train_input_length.npy')
training_img = np.load('/content/drive/My Drive/training_img.npy')
train_padded_txt = np.load('/content/drive/My Drive/train_padded_txt.npy')
max_label_len = np.load('/content/drive/My Drive/max_label_len.npy')

In [ ]:
print(train_label_length.shape)
print(train_input_length.shape)
print(training_img.shape)
print(train_padded_txt.shape)

(390000,)
(390000,)
(390000, 32, 128, 1)
(0, 0)


In [ ]:
inputs = Input(shape=(32,128,1))
s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)

# CNN layer
conv_1 = Conv2D(16, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(s)
conv_1 = Dropout(0.25)(conv_1)
conv_1 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_1)
# poolig layer
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

conv_2 = Conv2D(32, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(pool_1)
conv_2= BatchNormalization(axis=-1)(conv_2)
conv_2 = Dropout(0.25)(conv_2)
conv_2 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_2)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

conv_3 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(pool_2)
conv_3= BatchNormalization(axis=-1)(conv_3)
conv_3 = Dropout(0.25)(conv_3)
conv_3 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_3)
conv_4 = Conv2D(64, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_3)
# poolig layer
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

conv_5 = Conv2D(256, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)

conv_6 = Conv2D(256, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

act_model = Model(inputs, outputs)

act_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 128, 1)]      0         
                                                                 
 lambda (Lambda)             (None, 32, 128, 1)        0         
                                                                 
 conv2d (Conv2D)             (None, 32, 128, 16)       160       
                                                                 
 dropout (Dropout)           (None, 32, 128, 16)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 128, 32)       4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 64, 32)       0         
 )                                                               
                                                             

In [ ]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')

def ctc_lambda_func(args):
  y_pred, labels, input_length, label_length = args
  return K.ctc_batch_cost(labels, y_pred, input_length, label_length)

loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

ValueError: ignored

In [ ]:
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = keras.optimizers.Adam(lr=0.0005), metrics=['acc'])

filepath="/content/drive/My Drive/best_model.hdf5"
checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

In [ ]:
import matplotlib.pyplot as plt
def display(x,y):

  # load the saved best model weights
  act_model.load_weights('/content/drive/My Drive/best_model.hdf5')
  act_model.save('/content/drive/My Drive/act_model.hdf5')
  prediction = act_model.predict(x.reshape(1,32,128,1))

  # use CTC decoder
  out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
  x = x.reshape(32,128)
  plt.title('Input Image')
  plt.imshow(x)
  plt.axis('off')
  plt.show()

  # see the results
  for x in out:
      print("predicted text = ", end = '')
      for p in x:
          if int(p) != -1:
              print(char_list[int(p)], end = '')
      print('\n')

class DisplayCallback(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    global model
    model.save('/content/drive/My Drive/model.h5')
    i = np.random.randint(300000)
    display(training_img[i],train_padded_txt[i]

In [ ]:
%xmode Verbose

batch_size = 1026
epochs = 20

callbacks_list = [checkpoint, DisplayCallback()]

model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length]
          , y=np.zeros(len(training_img)),
          batch_size=batch_size,
          epochs = epochs,
          validation_split = 0.08,
          verbose = 1,
          callbacks = callbacks_list )

model.save("/content/drive/My Drive/best_model.hdf5")

In [ ]:
import matplotlib.pyplot as plt
from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional,Dropout
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import keras

In [ ]:
inputs = Input(shape=(32,128,1))
s = keras.layers.core.Lambda(lambda x: x / 255) (inputs)

# convolution layer with kernel size (3,3)
conv_1 = Conv2D(16, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(s)
conv_1 = Dropout(0.25)(conv_1)
conv_1 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_1)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)

conv_2 = Conv2D(32, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(pool_1)
conv_2= BatchNormalization(axis=-1)(conv_2)
conv_2 = Dropout(0.25)(conv_2)
conv_2 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_2)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)

conv_3 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(pool_2)
conv_3= BatchNormalization(axis=-1)(conv_3)
conv_3 = Dropout(0.25)(conv_3)
conv_3 = Conv2D(32, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_3)
conv_4 = Conv2D(64, (3,3), activation = 'relu', kernel_initializer='he_normal' ,padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)

conv_5 = Conv2D(256, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)

conv_6 = Conv2D(256, (3,3), activation = 'relu',kernel_initializer='he_normal' , padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)

squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)

# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)

outputs = Dense(62+1, activation = 'softmax')(blstm_2)

# model to be used at test time
act_model = Model(inputs, outputs)

act_model.summary()

In [ ]:
def display(x,y):

  # load the saved best model weights
  act_model.load_weights('/content/drive/My Drive/best_model.hdf5')
  prediction = act_model.predict(x.reshape(1,32,128,1))

  # use CTC decoder
  out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
  x = x.reshape(32,128)
  plt.title('Input Image')
  plt.imshow(x)
  plt.axis('off')
  plt.show()

  # see the results
  for x in out:
      print("predicted text = ", end = '')
      for p in x:
          if int(p) != -1:
              print(char_list[int(p)], end = '')
      print('\n')

In [ ]:
for _ in range(8):
  i = np.random.randint(300000)
  display(training_img[i],train_padded_txt[i])